In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [59]:
%matplotlib inline

In [60]:
data = pd.read_csv('Income_training.csv')
test = pd.read_csv('Income_testing.csv')

In [61]:
data.head()

,compositeHourlyWages,age,yearsEducation,sex1M0F
0,21.38,58,10,1
1,25.15,42,16,1
2,8.57,31,12,0
3,12.07,43,13,0
4,10.97,46,12,0


In [62]:
data.describe()

,compositeHourlyWages,age,yearsEducation,sex1M0F
count,3197.000000,3197.000000,3197.000000,3197.000000
mean,15.495127,36.884579,13.180794,0.491711
std,7.754763,11.996980,3.042127,0.500009
min,2.300000,16.000000,0.000000,0.000000
25%,9.250000,28.000000,12.000000,0.000000
50%,14.210000,36.000000,13.000000,0.000000
75%,19.650000,46.000000,15.000000,1.000000
max,49.920000,65.000000,20.000000,1.000000


In [63]:
data.isna().sum()

compositeHourlyWages    0
age                     0
yearsEducation          0
sex1M0F                 0
dtype: int64

In [64]:
data['catAge'] = pd.qcut(data.age,q=4,labels=False)
test['catAge'] = pd.qcut(test.age,q=4,labels=False)

In [65]:
data['catedu'] = pd.qcut(data.yearsEducation,q=4,labels=False)
test['catedu'] = pd.qcut(test.yearsEducation,q=4,labels=False)

In [66]:
trainX=data.drop(['compositeHourlyWages'],axis=1)
trainY=data['compositeHourlyWages']
testX = test.drop(['ID'],axis=1)

In [67]:
import sklearn
import xgboost as xgb

In [68]:
from sklearn.ensemble import RandomForestRegressor as rf

In [78]:
xgb1 = xgb.XGBRegressor(colsample_bytree=0.8,
                 gamma=0,                 
                 learning_rate=0.001,
                 max_depth=5,
                 min_child_weight=1,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.75,
                 cv=5,
                 seed=42) 

In [84]:
xgb1.fit(trainX,trainY,eval_metric='rmse')

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, cv=10, gamma=0, learning_rate=0.001,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=10000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1,
       seed=42, silent=True, subsample=0.75)

In [86]:
xgb1.feature_importances_
xgb1.evals_result

<bound method XGBModel.evals_result of XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, cv=10, gamma=0, learning_rate=0.001,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=10000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1,
       seed=42, silent=True, subsample=0.75)>

In [87]:
trainX.head()

,age,yearsEducation,sex1M0F,catAge,catedu
0,58,10,1,3,0
1,42,16,1,2,3
2,31,12,0,1,0
3,43,13,0,2,1
4,46,12,0,2,0


In [88]:
wage = xgb1.predict(testX)

In [89]:
wage

array([23.414495 , 19.199755 , 11.17465  , 19.495245 , 13.116385 ,
       19.091608 , 16.54247  , 19.70541  , 12.121746 , 17.076214 ,
       12.238341 ,  8.456589 ,  8.382847 , 12.80588  , 14.118797 ,
       20.002678 , 17.357723 ,  9.426744 , 12.934848 , 21.06505  ,
       15.493535 , 17.650356 , 14.511808 , 22.46886  , 13.214155 ,
       17.634344 , 18.59823  ,  8.382847 ,  6.4377327, 12.631687 ,
        6.967594 , 15.103856 , 22.539263 , 13.97384  , 11.369704 ,
       17.315422 , 13.337374 ,  8.865625 , 19.967443 , 17.083685 ,
       11.084749 , 19.967443 , 22.660025 ,  8.532091 , 15.046243 ,
       14.797942 , 10.888179 , 25.441296 , 21.323229 ,  7.7052703,
        7.7052703, 22.277607 , 14.331725 ,  6.595402 ,  8.752646 ,
       14.118797 , 21.641281 , 20.032911 , 17.504417 , 16.509308 ,
       21.017912 , 20.06996  , 13.932631 , 14.664213 , 12.396114 ,
        7.8427067, 22.277607 , 17.981794 , 13.079773 , 13.599446 ,
       13.649033 , 17.981794 , 14.605376 ,  8.456589 , 19.4726

In [90]:
test['compositeHourlyWages'] = pd.DataFrame(wage)

In [91]:
sub = test.drop(['age','yearsEducation','sex1M0F','catAge','catedu'],axis=1)

In [92]:
sub.to_csv('sub.csv',index=False)